## Dependancies

In [6]:
import xmltodict
import json
import re

## XML to dict

In [7]:
EventSubClass ={
    1: 'logical',
    2: 'physical'
}
queries = {}

with open('trace.xml', encoding='utf-16') as fd:
    queryPlan = xmltodict.parse(fd.read(), encoding='utf-16')

for event in queryPlan['TraceData']['Events']['Event']:
    if event['@name'] == 'DAX Query Plan':
        
        for column in event['Column']:
            
            if column['@name'] == 'TextData':
                query = column['#text']
    
            if column['@name'] == 'EventSubclass':
                queryType = EventSubClass[int(column['#text'])]
    
        queries[queryType] = query

## Generate Graph

In [28]:
def extract_records(line) -> int:
    match = re.search(r'#Records=(\d+)', line)
    if match:
        return int(match.group(1))
    return None

def extract_operation_type(line) -> list:
    match = re.search(r'(.*): (\w*)', line)
    if match:
        return [match.group(1).strip(), match.group(2).strip()]
    return None

def generate_graph(lines: list)->list:
    stack = []
    level_parents = {}
    graph = []

    for index, line in enumerate(lines):
        current_level = len(line) - len(line.lstrip())
        
        while len(stack) > current_level:
            stack.pop()
        
        parent_index = level_parents.get(current_level - 1, None)
        
        stack.append((index, line))
        
        level_parents[current_level] = index

        operationType = extract_operation_type(line)

        graph.append({
            'srcid': parent_index,
            'dstid': index,
            'operation': line.strip(),
            'operationShort': operationType[0],
            'operationType': operationType[1],
            'isCache': operationType[0] == 'Cache',
            'level': current_level,
            'records': extract_records(line)
        })
    
    return graph


In [29]:
graphs = {}

for queryType, query in queries.items():

    graphs[queryType] = generate_graph(lines = query.split('\n'))

with open('queryPlan.json', 'w') as fp:
    json.dump(graphs, fp)

RelLogOp
RelLogOp
RelLogOp
ScaLogOp
ScaLogOp
ScaLogOp
ScaLogOp
ScaLogOp
ScaLogOp
RelLogOp
RelLogOp
ScaLogOp
ScaLogOp
RelLogOp
RelLogOp
RelLogOp
RelLogOp
RelLogOp
ScaLogOp
ScaLogOp
ScaLogOp
ScaLogOp
ScaLogOp
ScaLogOp
ScaLogOp
IterPhyOp
IterPhyOp
SpoolPhyOp
IterPhyOp
IterPhyOp
IterPhyOp
IterPhyOp
SpoolPhyOp
IterPhyOp
IterPhyOp
SpoolPhyOp
IterPhyOp
IterPhyOp
SpoolPhyOp
IterPhyOp
LookupPhyOp
LookupPhyOp
SpoolPhyOp
IterPhyOp
SpoolPhyOp
IterPhyOp
IterPhyOp
SpoolPhyOp
IterPhyOp
IterPhyOp
IterPhyOp
IterPhyOp
SpoolPhyOp
IterPhyOp
IterPhyOp
SpoolPhyOp
IterPhyOp
LookupPhyOp
LookupPhyOp
LookupPhyOp
LookupPhyOp
LookupPhyOp
LookupPhyOp
LookupPhyOp
LookupPhyOp
